# Домашнее задание

LoRA и доменная адаптация

**Цель:**

В данном домашнем задании вам предстоит применять подход на основе LoRA для адаптации языковых моделей под различные задачи, дообучать языковые модели с помощью LoRA.

### Описание/Пошаговая инструкция выполнения домашнего задания:

1. Возьмите датасет для задачи саммаризации. Например, https://huggingface.co/datasets/EdinburghNLP/xsum или похожий по структуре на ваш выбор; для данного задания оставьте 10K-20K случайных примеров.

2. Выберите одну из генеративных моделей
   - https://huggingface.co/bigscience/bloom-1b1
   - https://huggingface.co/FacebookAI/roberta-large
    - https://huggingface.co/meta-llama/Llama-3.2-1B
    - https://huggingface.co/thepowerfuldeez/Qwen2-1.5B-Summarize

(если позволяют ресурсы, можно выбрать и другую модель бОльшего размера)

3. Подготовка данных
   - при необходимости предобработайте текст - обрежьте входные данные до лимита токена выбранной модели;
   - разделите данные на обучающую и валидационную выборки в соотношении 80/20;

4. Выберите метрику, по которой будете оценивать результаты обучения - ROUGE, BLEU, BERT score или что-то другое. Опишите мотивацию своего выбора.

5. Fine-tuning с помощью LoRA
>
А) используйте PEFT с методом LoRA или VB-LoRA
>
Б) сделайте несколько экспериментов обучения с разными гиперпараметрами
    - batch size: пара значений на ваш выбор, например 128 и 256
    - rank: два-три значения на ваш выбор, например 4, 16, 64

6. Оценка качества
   - выведите таблицу полученных метрик для каждой пары гиперпараметров
   - продемонстрируйте несколько предсказаний лучшей модели
   - если лучшие метрики показались неудовлетворительными, опишите, какие шаги можно было сделать по-другому, чтобы улучшить результат.


In [3]:
import os, gc

import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm, trange

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling ##DataCollatorWithPadding
from datasets import Dataset, DatasetDict, load_dataset

import peft
from peft import LoraConfig, get_peft_model, PeftModel

import evaluate

from torch.optim import AdamW
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

## 1. Выбор модели

В качестве модели выберем bigscience/bloomz-560m, модели большего размера в процессе обучения падают по out of memory на 24 Gb VRAM.

In [6]:
#base_model = "bigscience/bloom-1b1"
base_model = "bigscience/bloomz-560m"
#base_model="FacebookAI/roberta-large"

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [8]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer

BloomTokenizerFast(name_or_path='bigscience/bloomz-560m', vocab_size=250680, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [9]:
model = AutoModelForCausalLM.from_pretrained(base_model)

In [10]:
model.to(device)

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (

## 2. Подготовка даных

В качестве набора данных буду использовать [CNN / Daily Mail](https://huggingface.co/datasets/abisee/cnn_dailymail/tree/main/3.0.0) из предварительно загруженных локальных csv файлов. 

Попытка загрузки [Extreme Summarization (XSum) Dataset](https://huggingface.co/datasets/EdinburghNLP/xsum) или даже просто получение информации о наборе данных, приводит к ошибке:
> "UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc4 in position 4: invalid continuation byte"

In [12]:
path = "./data/cnn_dailymail/"
data_files={"train": path+"train.csv", "validation": path+"validation.csv", "test": path+"test.csv"}
data_files

{'train': './data/cnn_dailymail/train.csv',
 'validation': './data/cnn_dailymail/validation.csv',
 'test': './data/cnn_dailymail/test.csv'}

In [13]:
temp_datasets = load_dataset("csv", data_files=data_files, split=["train[:800]","validation[:200]"]) ## to get ~80/20 train/val split 

In [14]:
temp_datasets

[Dataset({
     features: ['id', 'article', 'highlights'],
     num_rows: 800
 }),
 Dataset({
     features: ['id', 'article', 'highlights'],
     num_rows: 200
 })]

In [15]:
## renaming and removing columns
for i in range(0, len(temp_datasets)):
    temp_datasets[i] = temp_datasets[i].rename_column("article","text")
    temp_datasets[i] = temp_datasets[i].rename_column("highlights", "target")
    temp_datasets[i] = temp_datasets[i].remove_columns("id")

In [16]:
temp_datasets

[Dataset({
     features: ['text', 'target'],
     num_rows: 800
 }),
 Dataset({
     features: ['text', 'target'],
     num_rows: 200
 })]

In [17]:
temp_datasets[0] = temp_datasets[0].map(lambda row: tokenizer(row['text']), batched=True)
temp_datasets[1] = temp_datasets[1].map(lambda row: tokenizer(row['text']), batched=True)

In [18]:
# Creating a single DatasetDict for easy access
data = DatasetDict({
    "train": temp_datasets[0],
    "validation": temp_datasets[1],
})

data

DatasetDict({
    train: Dataset({
        features: ['text', 'target', 'input_ids', 'attention_mask'],
        num_rows: 800
    })
    validation: Dataset({
        features: ['text', 'target', 'input_ids', 'attention_mask'],
        num_rows: 200
    })
})

In [19]:
data["train"][0]['text'], data["train"][0]['target']

("By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure. The diocese announced on Monday that Bishop John Folda is taking time off after being diagnosed with hepatitis A. The diocese says he contracted the infection through contaminated food while attending a conference for newly ordained

In [20]:
# Clearing memory from temporary datasets
del temp_datasets

## 4. Выбор метрики

- BLEU (Bilingual Evaluation Understudy) больше подходит, когда важно оценить точность (precision) перевода, как например, в задаче машинного перевода, для которой данная метрика и была создана изначально;
  
- ROUGE (Recall-Oriented Understudy for Gisting Evaluation) использует recall, precision и F-меру для рценки соответствия последовательностей слов (n-grams) между сгенерированным текстом и референсным написанным человеком текстом. Хорошо подходит для задач суммаризации текста, где цель - извлечь как можно больше ключевой информации из исходного документа.

- BERTscore использует контекстные эмбеддинги современных LLM, таких как BERT. Применяется для оценки семантической точности между сгенерированным и исходным текстом и подходит для решения широкого круга задач.

Будем использовать метрику ROUGE, как наиболее подходящую для задачи суммаризации.

In [23]:
rouge = evaluate.load('rouge')

## 5.Fine-tuning с помощью LoRA

Даже маленькая модель на 0.5B параметров на обучении поместилась в 24Gb VRAM только с batch_size=1. В связи с отсутствием возможности варьировать batch_size, протестируем модель на трех значениях ранга: 4, 16 и 64.

## 5.1 rank = 4

In [27]:
RANK = 4

In [28]:
lora_config = LoraConfig(
    r=RANK, # As bigger the R bigger the parameters to train.
    lora_alpha=1, # a scaling factor that adjusts the magnitude of the weight matrix. Usually set to 1
    target_modules=["query_key_value"], # You can obtain a list of target modules in the URL above.
    lora_dropout=0.05, # Helps to avoid Overfitting.
    bias="lora_only", # this specifies if the bias parameter should be trained.
    task_type="CAUSAL_LM"
)

In [29]:
peft_model = get_peft_model(model, lora_config)
print(peft_model.print_trainable_parameters())

trainable params: 466,944 || all params: 559,607,808 || trainable%: 0.0834
None


In [30]:
working_dir = './'
output_directory = os.path.join(working_dir, 'peft_lab_outputs')
output_directory

'./peft_lab_outputs'

In [31]:
# Creating the TrainingArgs
training_args = TrainingArguments(
    output_dir=output_directory,
    per_device_train_batch_size=1,  # Batch size per device for training
    per_device_eval_batch_size=1,   # Batch size per device for evaluating   
    #auto_find_batch_size=True, # Find a correct bvatch size that fits the size of Data.
    learning_rate= 3e-2, # Higher learning rate than full fine-tuning.
    num_train_epochs=4,
    #use_cpu=True
)

In [32]:
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=data["train"],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [33]:
trainer.train()

Step,Training Loss
500,12.853800
1000,9.538100
1500,8.604700
2000,8.284100
2500,8.101900
3000,8.063200


TrainOutput(global_step=3200, training_loss=9.188214950561523, metrics={'train_runtime': 375.7885, 'train_samples_per_second': 8.515, 'train_steps_per_second': 8.515, 'total_flos': 5102755699458048.0, 'train_loss': 9.188214950561523, 'epoch': 4.0})

In [34]:
# Save the model
peft_model_path = os.path.join(output_directory, f'lora_model')
trainer.model.save_pretrained(peft_model_path)

In [35]:
# Load the Model
peft_model = PeftModel.from_pretrained(model, peft_model_path, is_trainable=False)
peft_model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): Embedding(250880, 1024)
        (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0-23): 24 x BloomBlock(
            (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): lora.Linear(
                (base_layer): Linear(in_features=1024, out_features=3072, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=3072, bias=False)
                )
                (lora_e

In [36]:
gc.collect()
torch.cuda.empty_cache()

### Расчет метрики ROUGE для rank=4

In [38]:
results = {}
predictions = []
references = []

In [39]:
# this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=100):
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=max_new_tokens,
        repetition_penalty=1.5, # Avoid repetition.
        early_stopping=True, # The model can stop before reach the max_length
        eos_token_id=tokenizer.eos_token_id
    )
    return outputs

In [40]:
def quality(model, max_new_tokens=50):
    #predictions = []
    #references = []
    
    for item in tqdm(data["validation"]):
        input_ids = tokenizer(item['text'], return_tensors='pt', truncation=True, max_length=512).to(device)
        output_ids = get_outputs(model, input_ids, max_new_tokens=50)
        prediction = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        
        #Saving predictions and true labels 
        predictions.append(prediction)
        references.append(item['target'])
        
    # Calculating ROUGE metric
    rouge_dict = rouge.compute(predictions=predictions, references=references, use_stemmer=True)
    print('Evaluation:')
    for k, v in rouge_dict.items():
        print('\t{} = {:.5f}'.format(k, v))

    return [rouge_dict[k] for k in rouge_dict]    

In [41]:
results['rank='+f"{RANK}"] = quality(peft_model)

  0%|          | 0/200 [00:00<?, ?it/s]

Evaluation:
	rouge1 = 0.19643
	rouge2 = 0.10424
	rougeL = 0.13192
	rougeLsum = 0.16649


In [42]:
pd.DataFrame(results, index=['rouge1', 'rouge2', 'rougeL', 'rougeLsum']).T

,rouge1,rouge2,rougeL,rougeLsum
rank=4,0.196428,0.104236,0.131919,0.166488


In [43]:
# Example of prediction
predictions[0], len(predictions)

('Sally Forrest, an actress-dancer who graced the silver screen throughout the \'40s and \'50s in MGM musicals and films such as the 1956 noir While the City Sleeps died on March 15 at her home in Beverly Hills, California. Forrest, whose birth name was Katherine Feeney, was 86 and had long battled cancer. Her publicist, Judith Goffin, announced the news Thursday. Scroll down for video . Actress: Sally Forrest was in the 1951 Ida Lupino-directed film \'Hard, Fast and Beautiful\' (left) and the 1956 Fritz Lang movie \'While the City Sleeps\' A San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films including the critical and commercial success Not Wanted, Never Fear and Hard, Fast and Beautiful. Some of Forrest\'s other film credits included Bannerline, Son of Sinbad, and Excuse My Dust, according to her iMDB\xa0page. The page also indicates Forrest was in multiple Climax! and Rawhide television episodes. Forrest appeared a

In [44]:
# Reference example
references[0], len(references)

("Sally Forrest, an actress-dancer who graced the silver screen throughout the '40s and '50s in MGM musicals and films died on March 15 .\nForrest, whose birth name was Katherine Feeney, had long battled cancer .\nA San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films .",
 200)

## 5.2 rank=16

In [46]:
RANK = 16

In [47]:
lora_config = LoraConfig(
    r=RANK, # As bigger the R bigger the parameters to train.
    lora_alpha=1, # a scaling factor that adjusts the magnitude of the weight matrix. Usually set to 1
    target_modules=["query_key_value"], # You can obtain a list of target modules in the URL above.
    lora_dropout=0.05, # Helps to avoid Overfitting.
    bias="lora_only", # this specifies if the bias parameter should be trained.
    task_type="CAUSAL_LM"
)

In [48]:
del peft_model
peft_model = get_peft_model(model, lora_config)
print(peft_model.print_trainable_parameters())

trainable params: 1,646,592 || all params: 560,787,456 || trainable%: 0.2936
None


In [49]:
gc.collect()
torch.cuda.empty_cache()

In [50]:
training_args = TrainingArguments(
    output_dir=output_directory,
    per_device_train_batch_size=1,  # Batch size per device for training
    per_device_eval_batch_size=1,   # Batch size per device for evaluating   
    #auto_find_batch_size=True, # Find a correct bvatch size that fits the size of Data.
    learning_rate= 3e-2, # Higher learning rate than full fine-tuning.
    num_train_epochs=4,
    #use_cpu=True
)

In [51]:
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=data["train"],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
500,15.971000
1000,9.159400
1500,8.551300
2000,8.382700
2500,7.828300
3000,7.759400


TrainOutput(global_step=3200, training_loss=9.492246704101563, metrics={'train_runtime': 380.6493, 'train_samples_per_second': 8.407, 'train_steps_per_second': 8.407, 'total_flos': 5122641139040256.0, 'train_loss': 9.492246704101563, 'epoch': 4.0})

In [52]:
# Save the model
peft_model_path = os.path.join(output_directory, f'lora_model{RANK}')
trainer.model.save_pretrained(peft_model_path)

In [53]:
# Load the Model
peft_model = PeftModel.from_pretrained(model, peft_model_path, is_trainable=False)
peft_model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): Embedding(250880, 1024)
        (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0-23): 24 x BloomBlock(
            (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): lora.Linear(
                (base_layer): Linear(in_features=1024, out_features=3072, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora

In [54]:
gc.collect()
torch.cuda.empty_cache()

### Расчет метрики ROUGE для rank=16

In [56]:
predictions = []
references = []

In [57]:
results['rank='+f"{RANK}"] = quality(peft_model)

  0%|          | 0/200 [00:00<?, ?it/s]

Evaluation:
	rouge1 = 0.19650
	rouge2 = 0.10399
	rougeL = 0.13186
	rougeLsum = 0.16624


In [58]:
# Example of prediction
predictions[0], len(predictions)

('Sally Forrest, an actress-dancer who graced the silver screen throughout the \'40s and \'50s in MGM musicals and films such as the 1956 noir While the City Sleeps died on March 15 at her home in Beverly Hills, California. Forrest, whose birth name was Katherine Feeney, was 86 and had long battled cancer. Her publicist, Judith Goffin, announced the news Thursday. Scroll down for video . Actress: Sally Forrest was in the 1951 Ida Lupino-directed film \'Hard, Fast and Beautiful\' (left) and the 1956 Fritz Lang movie \'While the City Sleeps\' A San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films including the critical and commercial success Not Wanted, Never Fear and Hard, Fast and Beautiful. Some of Forrest\'s other film credits included Bannerline, Son of Sinbad, and Excuse My Dust, according to her iMDB\xa0page. The page also indicates Forrest was in multiple Climax! and Rawhide television episodes. Forrest appeared a

In [59]:
# Reference example
references[0], len(references)

("Sally Forrest, an actress-dancer who graced the silver screen throughout the '40s and '50s in MGM musicals and films died on March 15 .\nForrest, whose birth name was Katherine Feeney, had long battled cancer .\nA San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films .",
 200)

### 5.3 rank = 64

In [61]:
RANK = 64

In [62]:
lora_config = LoraConfig(
    r=RANK, # As bigger the R bigger the parameters to train.
    lora_alpha=1, # a scaling factor that adjusts the magnitude of the weight matrix. Usually set to 1
    target_modules=["query_key_value"], # You can obtain a list of target modules in the URL above.
    lora_dropout=0.05, # Helps to avoid Overfitting.
    bias="lora_only", # this specifies if the bias parameter should be trained.
    task_type="CAUSAL_LM"
)

In [63]:
del peft_model
peft_model = get_peft_model(model, lora_config)
print(peft_model.print_trainable_parameters())

trainable params: 6,365,184 || all params: 565,506,048 || trainable%: 1.1256
None


In [64]:
gc.collect()
torch.cuda.empty_cache()

In [65]:
training_args = TrainingArguments(
    output_dir=output_directory,
    per_device_train_batch_size=1,  # Batch size per device for training
    per_device_eval_batch_size=1,   # Batch size per device for evaluating   
    #auto_find_batch_size=True, # Find a correct bvatch size that fits the size of Data.
    learning_rate= 3e-2, # Higher learning rate than full fine-tuning.
    num_train_epochs=4,
    #use_cpu=True
)

In [66]:
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=data["train"],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
500,11.145900
1000,8.660600
1500,8.117900
2000,8.063100
2500,7.886900
3000,7.789200


TrainOutput(global_step=3200, training_loss=8.557541198730469, metrics={'train_runtime': 383.1796, 'train_samples_per_second': 8.351, 'train_steps_per_second': 8.351, 'total_flos': 5202182897369088.0, 'train_loss': 8.557541198730469, 'epoch': 4.0})

In [67]:
# Save the model
peft_model_path = os.path.join(output_directory, f'lora_model{RANK}')
trainer.model.save_pretrained(peft_model_path)

In [68]:
# Load the Model
peft_model = PeftModel.from_pretrained(model, peft_model_path, is_trainable=False)
peft_model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): Embedding(250880, 1024)
        (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0-23): 24 x BloomBlock(
            (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): lora.Linear(
                (base_layer): Linear(in_features=1024, out_features=3072, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=3072, bias=False)
                )
                (lora

In [69]:
gc.collect()
torch.cuda.empty_cache()

### Расчет метрики ROUGE для rank=64

In [71]:
predictions = []
references = []

In [72]:
results['rank='+f"{RANK}"] = quality(peft_model)

  0%|          | 0/200 [00:00<?, ?it/s]

Evaluation:
	rouge1 = 0.19630
	rouge2 = 0.10381
	rougeL = 0.13150
	rougeLsum = 0.16583


In [73]:
# Example of prediction
predictions[0], len(predictions)

("Sally Forrest, an actress-dancer who graced the silver screen throughout the '40s and '50s in MGM musicals and films such as the 1956 noir While the City Sleeps died on March 15 at her home in Beverly Hills, California. Forrest, whose birth name was Katherine Feeney, was 86 and had long battled cancer. Her publicist, Judith Goffin, announced the news Thursday. Scroll down for video . Actress: Sally Forrest was in the 1951 Ida Lupino-directed film 'Hard, Fast and Beautiful' (left) and the 1956 Fritz Lang movie 'While the City Sleeps' A San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films including the critical and commercial success Not Wanted, Never Fear and Hard, Fast and Beautiful. Some of Forrest's other film credits included Bannerline, Son of Sinbad, and Excuse My Dust, according to her iMDB\xa0page. The page also indicates Forrest was in multiple Climax! and Rawhide television episodes. Forrest appeared as herse

In [74]:
# Reference example
references[0], len(references)

("Sally Forrest, an actress-dancer who graced the silver screen throughout the '40s and '50s in MGM musicals and films died on March 15 .\nForrest, whose birth name was Katherine Feeney, had long battled cancer .\nA San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films .",
 200)

## 6. Сравнение результатов и выводы

Итоговая таблица результатов:

In [76]:
pd.DataFrame(results, index=['rouge1', 'rouge2', 'rougeL', 'rougeLsum']).T.sort_values(by=['rougeLsum'], ascending=False)

,rouge1,rouge2,rougeL,rougeLsum
rank=4,0.196428,0.104236,0.131919,0.166488
rank=16,0.196500,0.103992,0.131863,0.166245
rank=64,0.196301,0.103811,0.131501,0.165831


Результаты достаточно близкие и невысокие - для улучшения результатов, на мой взгляд, будет полезно:
- использовать больший объем данных для обучения модели;
- обучать в течении большего количества эпох;
- подобрать больший размер батча для повышения стабильности процесса обучения;

Реализация этих мер потребует уже более серьезных вычислительных ресурсов и времени.